In [9]:
import pandas as pd
import os
import numpy as np
import hashlib

In [10]:
notebook_dir = os.getcwd()

checkpoint4_csv_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint4_geocoded.csv')
checkpoint4 = pd.read_csv(checkpoint4_csv_path)

In [ ]:
# Define a function to generate a hashed ID
# Safe function to generate hashed ID
def generate_id(name, dob):
    if pd.isna(name) or pd.isna(dob):
        return ''
    combined = f"{str(name).strip()}_{str(dob).strip()}"
    return hashlib.sha256(combined.encode()).hexdigest()

# Apply the function only when 'Charges' exists
checkpoint4['person_id'] = checkpoint4.apply(
    lambda row: generate_id(row['Name'], row['DOB']) if pd.notna(row['Charges']) and str(row['Charges']).strip() != '' else '',
    axis=1
)

# Show the updated DataFrame
print(checkpoint4[checkpoint4['Name'].notna()][['Name', 'DOB', 'person_id']])

output_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint5_hashed.csv')
checkpoint4.to_csv(output_path, index=False)

                        Name         DOB  \
20451       AQUINO , ALFREDO  07/03/1974   
20452        SITHOLE , DAVID  01/21/1979   
20455         NUNEZ , MARTIN  06/26/2002   
20468      MARTINEZ , CARLOS  03/10/1987   
20470         TELLEZ , JAIRO  05/27/1994   
...                      ...         ...   
355494        PROULX , SHAUN  11/17/1985   
355542       LOPEZ , MICHAEL  11/22/1992   
355858  BETANCOURT , CHERLIE  01/18/1982   
356279         HOBBS , SCOTT  03/29/1971   
356691        ALSINA , ANGEL  06/25/1963   

                                                person_id  
20451   20f72481f083d4756c89b98fd499514c5953a8a4c10253...  
20452   2d5adb553cad8a22fd72d2e390525997c4963bac1b185e...  
20455   41c43f9f4255dee8e2c910e87f2a983e1b13beecf27eac...  
20468   6d5fb726029a990cafbca655fc265c2aa034d99d5ef62e...  
20470   bf5796a28c5e9c6a388dbc91b93e82176533fab08017c0...  
...                                                   ...  
355494  973beee58e1bb2c01531205bc042e8ce7ba3890645c